In [10]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

#dense layer
class Layer_Dense:

    #layer initialization
    def __init__(self,n_inputs,n_neurons):
        #initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1,n_neurons))

    #forward pass
    def forward(self,inputs):
        #calculate output values from inputs, weights, and biases
        self.output = np.dot(inputs,self.weights) + self.biases

class Activation_ReLU:

    #forward pass
    def forward(self,inputs):
        #calculate output values from input
        self.output = np.maximum(0,inputs)

class Activation_Softmax:

    #forward pass
    def forward(self,inputs):
        #get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
        #normalize them
        probabilities = exp_values / np.sum(exp_values,axis=1,keepdims=True)

        self.output = probabilities

class Loss:

    #calculates the data and regularization losses given model output and ground truth values
    def calculate(self,output,y):

        #calculate sample losses
        sample_losses = self.forward(output,y)

        #calculate mean loss
        data_loss = np.mean(sample_losses)

        #return loss
        return data_loss
    
class Loss_CategoricalCrossEntropy(Loss):

    def forward(self,y_pred,y_true):
        #number of samples in a batch
        samples = len(y_pred)

        #clip data to prevent division by 0
        #clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred,1e-7,1-1e-7)

        #probabilities for target values - onkly if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
        #mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true,axis = 1)
        
        #losses
        negative_log_likelihood = -np.log(correct_confidences)
        return negative_log_likelihood

In [12]:
#create dataset
X, y = spiral_data(samples=100,classes=3)

#create dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2,3)

#create ReLU activation
activation1 = Activation_ReLU()

#create second dense layer with 3 input features (as we take output of previous layer here) and 3 output values
dense2 = Layer_Dense(3,3)

#create softmax activation (to be used with dense layer):
activation2 = Activation_Softmax()

#create loss function
loss_function = Loss_CategoricalCrossEntropy()

#make forward pass of our training data through this layer
dense1.forward(X)

#make forward pass through activation
activation1.forward(dense1.output)

#make forward pass through second dense layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)
print(activation2.output[:5])

loss = loss_function.calculate(activation2.output,y)
print('loss:',loss)

[[0.33333334 0.33333334 0.33333334]
 [0.33333355 0.3333332  0.3333332 ]
 [0.33333382 0.33333313 0.3333331 ]
 [0.3333341  0.33333302 0.33333296]
 [0.33333433 0.3333329  0.33333278]]
loss: 1.0986081


In [9]:
softmax_output = np.array([[0.7,0.1,0.2],
                  [0.1,0.5,0.4],
                  [0.02,0.9,0.08]])
class_targets = np.array([0,1,1])

loss_function = Loss_CategoricalCrossEntropy()
loss = loss_function.calculate(softmax_output,class_targets)
print(loss)

0.38506088005216804
